In [1]:
import pandas as pd
import ta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import warnings
warnings.simplefilter("ignore")

In [2]:
load_dotenv()

True

In [3]:
api_key_pw = os.getenv('api_key_bot_IP')
api_secret_pw = os.getenv('api_secret_bot_IP')

In [4]:
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)

In [5]:
def get5minutedata(symbol, interval):
    frame = pd.DataFrame(session_auth.query_kline(symbol=symbol, interval=interval)["result"])
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [6]:
df = get5minutedata("ETHUSDT", "5m")
df

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-24 04:20:00,1637.36,1640.36,1636.80,1640.36,42.15284
2023-01-24 04:25:00,1640.36,1640.68,1637.08,1638.62,28.44273
2023-01-24 04:30:00,1638.62,1638.62,1636.97,1636.98,18.59142
2023-01-24 04:35:00,1636.98,1636.99,1636.12,1636.84,39.59181
2023-01-24 04:40:00,1636.84,1637.16,1636.51,1636.79,43.01813
...,...,...,...,...,...
2023-01-27 15:15:00,1588.55,1591.21,1582.95,1584.36,457.51559
2023-01-27 15:20:00,1584.36,1585.19,1581.00,1583.74,301.90594
2023-01-27 15:25:00,1583.74,1585.09,1581.90,1585.09,219.29544


In [7]:
eth = pd.read_csv ('./eth_actualizado.csv')
eth['Time'] = eth['time'].str.extract('(\d{4}-\d{2}-\d{2}\w\d{2}:\d{2}:\d{2})', expand=True)
eth['Time'] = pd.to_datetime(eth["Time"])
eth.set_index("Time", inplace = True)
eth.drop(columns="time", inplace=True)
eth.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
eth = eth[:7000]

In [8]:
eth

,Open,High,Low,Close,Volume
Time,,,,,
2022-12-19 01:00:00,1182.50,1183.10,1182.45,1182.50,1871.65
2022-12-19 01:05:00,1182.50,1182.90,1181.35,1182.60,3669.31
2022-12-19 01:10:00,1182.60,1186.10,1182.60,1185.30,6792.08
2022-12-19 01:15:00,1185.30,1186.25,1184.35,1186.15,2287.35
2022-12-19 01:20:00,1186.15,1188.80,1186.15,1188.55,3974.38
...,...,...,...,...,...
2023-01-12 07:55:00,1398.55,1398.60,1397.70,1397.70,1819.98
2023-01-12 08:00:00,1397.70,1398.60,1397.30,1397.35,2349.89
2023-01-12 08:05:00,1397.35,1397.35,1396.15,1396.90,1162.84


In [55]:
class ETH_Short(Strategy):

    def init(self):
        close = self.data.Close
        self.rsi= self.I(ta.momentum.rsi, pd.Series(close), window=14)
        self.K = self.I(ta.momentum.stochrsi_k, pd.Series(close), window= 14)
        self.D = self.I(ta.momentum.stochrsi_d, pd.Series(close), window= 14)

    def next(self):
        price = self.data.Close
        rsi = self.rsi
        K = self.K
        D = self.D
       
        if  rsi > 68 and K + 0.06 < D and D > 0.75:
            sl = price * 1.015
            tp = price * 0.97
            self.sell(sl=sl, tp=tp)
    

In [56]:
bt = Backtest(df, ETH_Short, cash= 10000, commission= 0.0015)

In [57]:
output = bt.run()

In [58]:
output

Start                     2023-01-24 04:20:00
End                       2023-01-27 15:35:00
Duration                      3 days 11:15:00
Exposure Time [%]                        61.8
Equity Final [$]                  9904.444345
Equity Peak [$]                   10109.18716
Return [%]                          -0.955557
Buy & Hold Return [%]               -3.219415
Return (Ann.) [%]                   -45.38684
Volatility (Ann.) [%]               20.325782
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.852039
Avg. Drawdown [%]                   -1.075047
Max. Drawdown Duration        2 days 00:25:00
Avg. Drawdown Duration        0 days 08:03:00
# Trades                                    4
Win Rate [%]                             25.0
Best Trade [%]                       2.854281
Worst Trade [%]                     -1.652479
Avg. Trade [%]                    

In [59]:
bt.plot()

Row(id='11061', ...)